### Техники оптимизации С++ программ.

Выдать домашнее задание по оптимизациям

<br />

##### Уровни оптимизации компилятора

gcc / clang:

https://gcc.gnu.org/onlinedocs/gcc/Optimize-Options.html

* `-O0` - без оптимизаций (для отладки)
* `-O1` - набор оптимизаций 1
* `-O2` - набор оптимизаций 1 + набор оптимизаций 2
* `-O3` - набор оптимизаций 1 + набор оптимизаций 2 + набор оптимизаций 3

Чем выше `On`, тем больше время компиляции, но эффективнее код
(на самом деле it depends)

Дополнительные варианты:
* `-Os` - сфокусироваться на оптимизации размера бинарного файла вместо скорости выполнения
* `-Ofast` = `-O3` + `-ffast-math` - ослабить требования к математическим вычислениям - не выполнять все требования IEEE (может ускорить вычислительные алгоритмы, но программист должен поклясться на крови полуночных русалок, что урезанных требований и меньшей точности достаточно)

Закинуть это код в godbolt, показать какой ассемблер генерирует компилятор для последнего gcc и clang на разных уровнях оптимизаций:

```c++
#include <vector>

int sum(const std::vector<int>& x)
{
    int rv = 0;
    for (int v : x)
        rv += v;
    return rv;
}
```

<br />

msvc:

https://docs.microsoft.com/en-us/cpp/build/reference/o-options-optimize-code?view=vs-2019

Аналоги: `/Od`, `/O1`, `/O2`, `/Os` + доп. варинты (см. ссылку)

<br />

##### Профилировка

**msvc:** Показать пример профилировки и просмотра результатов на msvc

<br />

**встроенный профилировщик в clang/gcc:**

Скрипт запуска встроенного в clang/gcc профилировщика:

```sh
echo "cleanup"
rm -f gmon.out
rm -f analysis.txt
rm -f a.out

echo "compile"
clang++-8 -pg -O2 reference.cpp -stdlib=libc++ -std=c++17

echo "running"
./a.out

echo "analyze"
gprof a.out gmon.out > analysis.txt
```

<br />

**google perf:**

Предустановки для инструмента google perf:

```sh
sudo apt-get install google-perftools libgoogle-perftools-dev
sudo apt-get install kcachegrind
```

Прогон google perf:

```sh
echo "cleanup"
rm -f a.out
rm -f cpu_profile
rm -f callgrind

echo "compile"
clang++-8 -O2 reference.cpp -std=c++17 -stdlib=libc++

echo "run with profile"
LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libprofiler.so CPUPROFILE=cpu_profile CPUPROFILE_FREQUENCY=5000 ./a.out

echo "analyze"
# google-pprof --gv   a.out cpu_profile  # X
# google-pprof --web  a.out cpu_profile  # browser
# google-pprof --test a.out cpu_profile  # console
google-pprof --callgrind a.out cpu_profile > callgrind && kcachegrind callgrind
```

<br />

##### Правила оптимизации:

1. Убедитесь, что оптимизировать действительно надо.
2. Напишите performance-тест для измерения производительности
3. Одна гипотеза - одно измерение
4. Отпрофилировать и найти самое тяжёлое место
5. Оптимизация сверху вниз (ни в коем случае не спускаться на ступеньку ниже, пока не поймём, что возможности ступеньки выше исчерпаны)
  * Нужны ли эти вычисления? (нет - убираем / кешируем - работает примитивная логика)
  * Дело в асимптотике алгоритма? (да - меняем алгоритм - работает теория алгоритмов)
  * Можно ли ускорить на уровне абстракций С++? (примитивные примеры: `reserve`, `make_shared`, `std::string::append`, `mutex -> atomic` etc - работает знание основ языка)
  * Можно ли ускорить на уровне абстракций ОС? (примитивные примеры: virtual memory organization и разбиение на процессы - работает знание ОС - не в рамках этого курса)
  * Welcome to michroarchitecture optimization guide! (примитивные примеры: strict aliasing, loops unrolling, vectorization etc. - вопрос, скорее всего, не будет покрываться в рамках курса)

<br />

##### Модель памяти

![](mem_hierarchy_single_cpu.png)

<br />

![](mem_hierarchy_multi_cpu.png)

Дополнительно рассказать про кешлайны, многопоточность и разреженность

<br />

**Замечания после семинара**:

* На семинаре успели только быстренько пройтись по домашнему заданию и посмотреть на скрипт как профилировать. Нужно ещё додать про профилирование и рассказать про "модель памяти", которая про кеши-регистры